Please leave this here and change the time 

In [1]:
print("Hello World!!")

Hello World!


Data Scraping


In [2]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Create a single DataFrame to store all seasons
all_data = pd.DataFrame()

for season in range(2002, 2025):
    url = f"https://www.espn.com/nba/stats/player/_/season/{season}/seasontype/2"
    driver.get(url)
    time.sleep(4)  # Allow JS to render

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Grab both stat tables
    tables = soup.find_all("table")
    if len(tables) < 2:
        print(f"⚠️ Skipping {season}: Tables not found")
        continue

    left_table = tables[0]
    right_table = tables[1]

    # Extract headers and rows
    left_headers = [th.text.strip() for th in left_table.find_all("th")]
    right_headers = [th.text.strip() for th in right_table.find_all("th")]

    left_rows = [[td.text.strip() for td in tr.find_all("td")] for tr in left_table.find_all("tr")[1:]]
    right_rows = [[td.text.strip() for td in tr.find_all("td")] for tr in right_table.find_all("tr")[1:]]

    # Skip if row mismatch
    if len(left_rows) != len(right_rows):
        print(f"⚠️ Skipping {season}: Row mismatch")
        continue

    # Combine and append
    combined = [l + r for l, r in zip(left_rows, right_rows)]
    df = pd.DataFrame(combined, columns=left_headers + right_headers)
    df["season"] = season
    all_data = pd.concat([all_data, df], ignore_index=True)
    print(f"✅ Scraped {season}")

driver.quit()

# Save all seasons to SQLite
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")
all_data.to_sql("player_season_stats", conn, if_exists="replace", index=False)
conn.close()

print("🏀 All seasons saved to SQLite successfully!")


✅ Scraped 2002
✅ Scraped 2003
✅ Scraped 2004
✅ Scraped 2005
✅ Scraped 2006
✅ Scraped 2007
✅ Scraped 2008
✅ Scraped 2009
✅ Scraped 2010
✅ Scraped 2011
✅ Scraped 2012
✅ Scraped 2013


KeyboardInterrupt: 

Single Season extraction:

In [ ]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Create a single DataFrame to store all seasons
all_data = pd.DataFrame()

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode (no GUI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Go to ESPN stats page
season = 2025
url = f"https://www.espn.com/nba/stats/player/_/season/{season}/seasontype/2"
driver.get(url)
time.sleep(5)  # wait for JS to render

# Parse rendered HTML
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Get both visible tables
tables = soup.find_all("table")
if len(tables) < 2:
    raise Exception("Could not find both stat tables")

left_table = tables[0]
right_table = tables[1]

# Extract headers
left_headers = [th.text.strip() for th in left_table.find_all("th")]
right_headers = [th.text.strip() for th in right_table.find_all("th")]

# Extract rows
left_rows = [[td.text.strip() for td in tr.find_all("td")] for tr in left_table.find_all("tr")[1:]]
right_rows = [[td.text.strip() for td in tr.find_all("td")] for tr in right_table.find_all("tr")[1:]]

# Combine the two parts
combined = []
for lrow, rrow in zip(left_rows, right_rows):
    combined.append(lrow + rrow)

# Final DataFrame
all_headers = left_headers + right_headers
df = pd.DataFrame(combined, columns=all_headers)
df["season"] = season

# Preview
print(df.head())


# Save to SQLite
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")

# Save or append to the player_season_stats table
df.to_sql("player_season_stats", conn, if_exists="replace", index=False)

conn.close()
print("✅ Full stats saved to SQLite!")


  RK                        Name POS  GP   MIN   PTS   FGM   FGA   FG%  3PM  \
0  1  Shai Gilgeous-AlexanderOKC  PG  76  34.2  32.7  11.3  21.8  51.9  2.1   
1  2    Giannis AntetokounmpoMIL  PF  67  34.2  30.4  11.8  19.7  60.1  0.2   
2  3             Nikola JokicDEN   C  70  36.7  29.6  11.2  19.5  57.6  2.0   
3  4          Anthony EdwardsMIN  SG  79  36.3  27.6   9.1  20.4  44.7  4.1   
4  5             Jayson TatumBOS  SF  72  36.4  26.8   9.2  20.3  45.2  3.5   

   ...   FTA   FT%   REB   AST  STL  BLK   TO DD2 TD3 season  
0  ...   8.8  89.8   5.0   6.4  1.7  1.0  2.4   6   0   2025  
1  ...  10.6  61.7  11.9   6.5  0.9  1.2  3.1  55  11   2025  
2  ...   6.4  80.0  12.7  10.2  1.8  0.6  3.3  59  34   2025  
3  ...   6.3  83.7   5.7   4.5  1.2  0.6  3.2   7   0   2025  
4  ...   6.1  81.4   8.7   6.0  1.1  0.5  2.9  31   2   2025  

[5 rows x 23 columns]
✅ Full stats saved to SQLite!


In [ ]:
# import os

# folder_path = "../data/sqlite"
# print("Exists:", os.path.exists(folder_path))
# print("Writable:", os.access(folder_path, os.W_OK))
# print("Current working directory:", os.getcwd())


Exists: True
Writable: True
Current working directory: c:\Users\priya\Documents\GitHub\NBA_MVP_Predictor\notebooks


This code will clear the player table (Good while populating the database, and facing errors)

In [ ]:
import sqlite3

# Connect to the existing database
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")
cursor = conn.cursor()

# Drop the table if it exists
cursor.execute("DROP TABLE IF EXISTS player_season_stats")

conn.commit()
conn.close()

print("✅ Existing table dropped.")


✅ Existing table dropped.


This code will print the first 10 rows the the player_stats table in the database

In [9]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")

# View table names
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql(query_tables, conn)
print("Tables in database:")
print(tables_df)

# Read the entire table (you can limit this if the table is huge)
query = "SELECT * FROM player_season_stats;"
df = pd.read_sql(query, conn)

# Show sample
print("\nSample data from table:")
print(df.head())

# Show shape (rows and columns)
print(f"\n📊 Table shape: {df.shape[0]} rows × {df.shape[1]} columns")

conn.close()


Tables in database:
                  name
0  player_season_stats

Sample data from table:
  RK                 Name POS  GP   MIN   PTS   FGM   FGA   FG%  3PM  ...  \
0  -     Allen IversonPHI  SG  60  43.7  31.4  11.1  27.8  39.8  1.3  ...   
1  -  Shaquille O'NealLAL   C  67  36.1  27.2  10.6  18.3  57.9  0.0  ...   
2  -       Paul PierceBOS  SF  82  40.3  26.1   8.6  19.5  44.2  2.6  ...   
3  -     Tracy McGradyORL  SG  76  38.3  25.6   9.4  20.9  45.1  1.4  ...   
4  -         Tim DuncanSA   C  82  40.6  25.5   9.3  18.3  50.8  0.0  ...   

    FTA   FT%   REB  AST  STL  BLK   TO DD2 TD3 season  
0   9.8  81.2   4.5  5.5  2.8  0.2  4.0   4   1   2002  
1  10.7  55.5  10.7  3.0  0.6  2.0  2.6  40   0   2002  
2   7.8  80.9   6.9  3.2  1.9  1.0  2.9  17   0   2002  
3   7.3  74.8   7.9  5.3  1.6  1.0  2.5  24   1   2002  
4   8.5  79.9  12.7  3.7  0.7  2.5  3.2  67   0   2002  

[5 rows x 23 columns]

📊 Table shape: 1150 rows × 23 columns


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

# URL of ESPN MVP history page
url = "https://www.espn.com/nba/history/awards/_/id/33"

# Send request
headers = {"User-Agent": "Mozilla/5.0"}
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

# Find all rows in the MVP awards table
tables = soup.find_all("table")
mvp_table = tables[0]  # MVP is usually the first table

data = []
for row in mvp_table.find_all("tr")[1:]:  # Skip header row
    cols = row.find_all("td")
    if len(cols) >= 2:
        year = cols[0].text.strip()
        player = cols[1].text.strip()

        # Only keep years from 2002 to 2025
        if year.isdigit() and 2002 <= int(year) <= 2025:
            data.append((int(year), player))

# Convert to DataFrame
df_mvp = pd.DataFrame(data, columns=["Year", "Player"])
print(df_mvp)

# Save as Table 

conn = sqlite3.connect("../data/sqlite/nba_mvp.db")
df_mvp.to_sql("mvp_list", conn, if_exists="replace", index=False)
conn.close()


    Year                   Player
0   2025  Shai Gilgeous-Alexander
1   2024             Nikola Jokic
2   2023              Joel Embiid
3   2022             Nikola Jokic
4   2021             Nikola Jokic
5   2020    Giannis Antetokounmpo
6   2019    Giannis Antetokounmpo
7   2018             James Harden
8   2017        Russell Westbrook
9   2016            Stephen Curry
10  2015            Stephen Curry
11  2014             Kevin Durant
12  2013             LeBron James
13  2012             LeBron James
14  2011             Derrick Rose
15  2010             LeBron James
16  2009             LeBron James
17  2008              Kobe Bryant
18  2007            Dirk Nowitzki
19  2006               Steve Nash
20  2005               Steve Nash
21  2004            Kevin Garnett
22  2003               Tim Duncan
23  2002               Tim Duncan
